# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#csv exported in Part I to a DataFrame
weather_vac = "../output_data/new_city_weather.csv"
weather_data = pd.read_csv(weather_vac)
weather_data

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,0,Ribeira Grande,0,PT,1596434400,84,71.82,38.5167,-28.7000,11.79
1,1,Berlevåg,95,NO,1596434400,90,48.31,70.8578,29.0864,9.53
2,2,Punta Arenas,100,CL,1596434400,76,39.90,-53.1500,-70.9167,19.62
3,3,Mnogovershinnyy,97,RU,1596434400,69,56.91,53.9353,139.9242,7.67
4,4,Bathsheba,66,BB,1596434400,80,81.59,13.2167,-59.5167,12.01
...,...,...,...,...,...,...,...,...,...,...
543,543,Zhangjiakou Shi Xuanhua Qu,1,CN,1596434400,30,87.93,40.6103,115.0447,13.62
544,544,Usinsk,99,RU,1596434400,93,55.58,65.9939,57.5281,11.81
545,545,Harwich,100,GB,1596434400,81,58.78,51.9419,1.2844,12.68
546,546,Lusambo,59,CD,1596434400,40,78.91,-4.9667,23.4500,2.13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

In [4]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[["Lat", "Lng"]].astype(float)
hum=weather_data["Humidity"].astype(float)

In [5]:
# Create a Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("../Images/vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
first = weather_data.loc[weather_data["Temp"]<80,:]
second = first.loc[weather_data["Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
366,366,Yerbogachën,0,RU,1596434400,40,76.95,61.2767,108.0108,7.72
387,387,Jalu,0,LY,1596434400,70,79.32,29.0331,21.5482,9.89
394,394,Taywarah,0,AF,1596434400,25,73.49,33.3500,64.4167,4.54
439,439,Fuerte Olimpo,0,PY,1596434400,47,70.84,-21.0415,-57.8738,4.07
445,445,Bījār,0,IR,1596434400,25,78.19,35.8668,47.6051,5.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Lat']
hotel_df['Lng'] = fifth['Lng']

hotel_df.head()

,City,Country,Lat,Lng
366,Yerbogachën,RU,61.2767,108.0108
387,Jalu,LY,29.0331,21.5482
394,Taywarah,AF,33.3500,64.4167
439,Fuerte Olimpo,PY,-21.0415,-57.8738
445,Bījār,IR,35.8668,47.6051


In [8]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 366: Yerbogachën.
Closest hotel to Yerbogachën at 61.2767 , 108.0108 is Katangskaya Rb.
------------
Retrieving Results for Index 387: Jalu.
Closest hotel to Jalu at 29.0331 , 21.5482 is رؤيه للبصريات والساعات.
------------
Retrieving Results for Index 394: Taywarah.
Missing field/result... skipping.
------------
Retrieving Results for Index 439: Fuerte Olimpo.
Closest hotel to Fuerte Olimpo at -21.0415 , -57.8738 is Hidrocarburos del Norte.
------------
Retrieving Results for Index 445: Bījār.
Closest hotel to Bījār at 35.8668 , 47.6051 is هیئت خادم العباس(ع).
------------


In [9]:
#Add a "Hotel Name" column to the DataFrame.
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
366,Yerbogachën,RU,61.2767,108.0108,Katangskaya Rb
387,Jalu,LY,29.0331,21.5482,رؤيه للبصريات والساعات
394,Taywarah,AF,33.3500,64.4167,NaN
439,Fuerte Olimpo,PY,-21.0415,-57.8738,Hidrocarburos del Norte
445,Bījār,IR,35.8668,47.6051,هیئت خادم العباس(ع)


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
plt.savefig("../Images/gmapmarker.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>